In [50]:
import os
from collections import defaultdict, OrderedDict
os.chdir("../data")

In [88]:
# 读入数据
with open("translate/valid_ques.txt", "r") as f:
     valid_ques_ner = [txt.strip().split('\t') for txt in f.readlines()]
        
with open("translate/train_ques.txt", "r") as f:
    train_ques_sols = [txt.strip().split('\t') for txt in f.readlines()]

with open("translate/triples.txt", "r") as f:
    triples = [txt.strip().split('\t') for txt in f.readlines()]

### 构建子图

In [138]:
"字符转换"
_shift = lambda st:st.replace(' ', '_')

def nocycle(sol):
    """子图是否带圈"""
    objs = [obj for i, obj in enumerate(sol) if i % 2 == 0]
    return len(objs) == len(set(objs))

# 构建子图
def construct_subgraph(sub, edges, jump=3):
    """
    - 构造以 sub 为顶点的3/4跳子图
    - 返回按层得到的列表(BFS)
    - 特别注意：edges 中空格为 _
    """
    next_level, res = [(_shift(sub),)], []
    for _ in range(jump):
        level, next_level = next_level, []
        for edge in level:
            s = edge[-1]
            if s not in edges:continue
            # 下一层节点(保留路径)
            next_level.extend(edge + newedge for newedge in edges[s]) 
        res.append([path for path in next_level if nocycle(path)])
    return res

### 处理验证集

In [125]:
# 构建边集（**必要时添加反向关系**）
edges = defaultdict(list)
for sub, rel, obj in triples:
    edges[sub].append((rel, obj))
    edges[obj].append((rel, sub))
sort_edges = list(sorted(edges.items(), key=lambda i:-len(i[-1])))

4225

In [139]:
# 不存在两跳的数据集（哪怕反向）
nonsol = []
subgraphs = {}
for que, ner in valid_ques_ner: # 存在部分重复问题，这部分会被覆盖
    subgraphs[que] = construct_subgraph(ner, edges)
    if not len(subgraphs[que][1]):
        nonsol.append(que)
len(nonsol)

124

In [140]:
# 构建边集（**必要时添加反向关系**）
edges = defaultdict(list)
for sub, rel, obj in triples:
    edges[sub].append((rel, obj))
sort_edges = list(sorted(edges.items(), key=lambda i:-len(i[-1])))
len(sort_edges[0][1]) # 143 条边，然而仅有一跳

143

In [141]:
# 不存在两跳的数据集（哪怕反向）
nonsol = []
subgraphs = {}
for que, ner in valid_ques_ner: # 存在部分重复问题，这部分会被覆盖
    subgraphs[que] = construct_subgraph(ner, edges)
    if not len(subgraphs[que][1]):
        nonsol.append(que)
len(nonsol)

309